<a href="https://colab.research.google.com/github/Lneves23/10xDataAnalysisWorkshop/blob/main/CoLab_Integrating_Single_Cell_and_Visium_Spatial_GEX_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integrating Single Cell and Visium Spatial Gene Expression Data



2024


Uses spacexr, an R-based community developed tool, to integrate paired single-cell and spatial datasets: https://github.com/dmcable/spacexr

Code is based on the 10x Genomics Analysis Guide 'Integrating 10x Visium and Chromium Data with R': https://www.10xgenomics.com/analysis-guides/integrating-10x-visium-and-chromium-data-with-r

# Install packages

**Step 1:**

Adjust Ubuntu parameters for downloading R packages.

*This is only required for Google CoLabs. This is not needed for use in RStudio.*

In [ ]:
system('apt-get update')
system('sudo apt-get install libudunits2-dev libgdal-dev libgeos-dev libproj-dev libglpk-dev libmagick++-dev libgsl-dev')

**Step 2:**

Install and load necessary packages for Seurat and SpaceXR.

*This can take approximately 25 minutes.*



In [ ]:
if (!require("remotes")) install.packages("remotes")
if (!require("hdf5r")) install.packages("hdf5r")
if (!require("Seurat")) install.packages("Seurat")
if (!require("ggplot2")) install.packages("ggplot2")
devtools::install_github("dmcable/spacexr", build_vignettes = FALSE)

**Step 3:**

Load Seurat, SpaceXR, and other necessary packages.

In [ ]:
library(spacexr)
library(Seurat)
library(ggplot2)
library(hdf5r)


# Download single-cell dataset




**Step 4:**

1.   Download matrix data (.h5) to integrate from this dataset: https://www.10xgenomics.com/datasets/mixture-of-colorectal-cancer-and-healthy-kidney-ffpe-tissues-dissociated-manually-or-using-gentlemacs-dissociator-multiplexed-samples-4-probe-barcodes-1-standard

2.   Read in and create a Seurat object from single-cell matrix.

In [ ]:
download.file("https://cf.10xgenomics.com/samples/cell-exp/7.1.0/4plex_human_colorectal_kidney_scFFPE_multiplex_Colorectal_Octo_BC2/4plex_human_colorectal_kidney_scFFPE_multiplex_Colorectal_Octo_BC2_count_sample_filtered_feature_bc_matrix.h5","Colorectal_Cancer_Matrix.h5")

Counts <- Read10X_h5("Colorectal_Cancer_Matrix.h5")
count_SeuratObject <- CreateSeuratObject(Counts)
sc_counts <- count_SeuratObject[["RNA"]]$counts

**Step 5:**

Download cell annotations file from Loupe Browser

In [ ]:
celltype <- read.csv("https://raw.githubusercontent.com/Lneves23/10xDataAnalysisWorkshop/refs/heads/main/Spatial%20Integration/CellTypes_fromLoupe.csv")
celltype <- setNames(celltype[[2]], celltype[[1]])
table(celltype)
celltype <- as.factor(celltype)

# Download Visium dataset

**Step 6:**

1.   Download Visium data output package:https://www.10xgenomics.com/datasets/human-colorectal-cancer-11-mm-capture-area-ffpe-2-standard

2.   Read in and create a Seurat Visium Spatial object from output folder from SpaceRanger

Generates spatial plot of UMI counts

In [ ]:
system("mkdir ColorectalCancerVisiumOuts")
download.file("https://cf.10xgenomics.com/samples/spatial-exp/2.0.1/CytAssist_11mm_FFPE_Human_Colorectal_Cancer/CytAssist_11mm_FFPE_Human_Colorectal_Cancer_filtered_feature_bc_matrix.h5", "ColorectalCancerVisiumOuts/filtered_feature_bc_matrix.h5")
download.file("https://cf.10xgenomics.com/samples/spatial-exp/2.0.1/CytAssist_11mm_FFPE_Human_Colorectal_Cancer/CytAssist_11mm_FFPE_Human_Colorectal_Cancer_filtered_feature_bc_matrix.tar.gz", "ColorectalCancerVisiumOuts/filtered_feature_bc_matrix.tar.gz")
download.file("https://cf.10xgenomics.com/samples/spatial-exp/2.0.1/CytAssist_11mm_FFPE_Human_Colorectal_Cancer/CytAssist_11mm_FFPE_Human_Colorectal_Cancer_analysis.tar.gz", "ColorectalCancerVisiumOuts/analysis.tar.gz")
download.file("https://cf.10xgenomics.com/samples/spatial-exp/2.0.1/CytAssist_11mm_FFPE_Human_Colorectal_Cancer/CytAssist_11mm_FFPE_Human_Colorectal_Cancer_spatial.tar.gz", "ColorectalCancerVisiumOuts/spatial.tar.gz")
download.file("https://cf.10xgenomics.com/samples/spatial-exp/2.0.1/CytAssist_11mm_FFPE_Human_Colorectal_Cancer/CytAssist_11mm_FFPE_Human_Colorectal_Cancer_metrics_summary.csv", "ColorectalCancerVisiumOuts/metrics_summary.csv")
untar("ColorectalCancerVisiumOuts/spatial.tar.gz", exdir = "ColorectalCancerVisiumOuts")
VisiumData <- read.VisiumSpatialRNA("ColorectalCancerVisiumOuts")
barcodes <- colnames(VisiumData@counts)

plot_puck_continuous(puck=VisiumData, barcodes=barcodes, plot_val=VisiumData@nUMI,
                     size=1, ylimit=c(0,round(quantile(VisiumData@nUMI,0.9))),
                     title='plot of nUMI')

**Step 7**

Adjust coordinates to match H&E image
Scale projected image to original H&E image

In [ ]:
vis_coords <- read.csv("ColorectalCancerVisiumOuts/spatial/tissue_positions.csv", header = TRUE)
vis_coords[,7] <- vis_coords[,3]
vis_coords[,3] <- vis_coords[,4]
vis_coords[,4] <- vis_coords[,7]
vis_coords <- vis_coords[,1:6]
vis_coords[,4] <- vis_coords[,4]*1.756

write.table(vis_coords, "ColorectalCancerVisiumOuts/spatial/tissue_positions.csv", quote=FALSE, row.names=FALSE, sep=',')

VisiumData <- read.VisiumSpatialRNA("ColorectalCancerVisiumOuts")

plot_puck_continuous(puck=VisiumData, barcodes=barcodes, plot_val=VisiumData@nUMI,
                     size=1, ylimit=c(0,round(quantile(VisiumData@nUMI,0.9))),
                     title='plot of nUMI')

#Create a reference file from Chromium single-cell data

**Step 8:**

1.   Create matrices using gene and UMI counts from loaded Chromium dataset.
2.   Generate a reference file from gene counts, umis, and cell types



In [ ]:
sc_counts <- count_SeuratObject[["RNA"]]$counts

sc_umis <- count_SeuratObject@meta.data[,c(1,2)]
sc_umis <- setNames(sc_umis[[2]], rownames(sc_umis))

SCreference <- Reference(sc_counts, celltype, sc_umis)

# Visium spot deconvolution and cell type annotation

Use SpaceXR to run robust cell type deconvolution.

See 10x Genomics Analysis Guide for more: https://www.10xgenomics.com/analysis-guides/integrating-single-cell-and-visium-spatial-gene-expression-data

**Step 9:**

Run SpaceXR to deconvolute Visium spots and assign cell types using the single-cell reference file from Step 6 to deconvolute Visium spots and assign cell types.

In [ ]:
myRCTD <- create.RCTD(VisiumData, SCreference, max_cores = 2)
myRCTD <- run.RCTD(myRCTD, doublet_mode = "full")

# Extract barcodes and weighted cell types for each spot, then normalize
barcodes <- colnames(myRCTD@spatialRNA@counts)
weights <- myRCTD@results$weights
norm_weights <- normalize_weights(weights)
cell_type_names<-colnames(norm_weights)

**Step 10:**

Generate spatial plots with annotated cell types

In [ ]:
resultsdir <- "RCTD_output_plots/"
dir.create(resultsdir)

#plot all cell types as a bar graph
spatialRNA <- myRCTD@spatialRNA
plot_cond_occur(cell_type_names, resultsdir, norm_weights, spatialRNA) + theme(axis.text.x = element_text(angle = 45, vjust = 1, hjust=1))
ggsave(paste(resultsdir,"celltype_plot.jpg", sep=""))


#plot BRCA1+ cells
plot_puck_continuous(myRCTD@spatialRNA, barcodes, norm_weights[,"BRCA1+ epithelial"], title = "BRCA1+ epithelial", size=1)
ggsave(paste(resultsdir,"BRCA1pos_cells.jpg", sep=""))

#plot B cells
plot_puck_continuous(myRCTD@spatialRNA, barcodes, norm_weights[,"fibroblast"], title = "Fibroblasts", size=1)
ggsave(paste(resultsdir,"Fibroblasts.jpg", sep=""))
